In [1]:
from cobra.io import read_sbml_model
from pathlib import Path
import os

# change working directory to context_specific_models folder
os.chdir('C:/Users/prins/git/Human1_RPE-PR') 

# create a dictionary with the filenames (keys) and models (values)
folder = Path().cwd()
files = [f for f in os.listdir(folder / 'cs_mods') if '.xml'in f]
models = {f:read_sbml_model(folder / 'cs_mods' / f ) for f in files}
k = list(models.keys())

# add model names as IDs
models_names = ['Human1', 'Schwan (1)', 'Schwan (2)', 'Melanocytes', 'Endothelial','Muscle','Fibroblasts',\
                'RPE','B cells','T + NK cells', 'Monocytes + Macrophages','Mast','Retina','RPE + Choroid',\
                'PR rod','Amacrine','Bipolar','PR cone', 'Horizontal', 'Müller','Ganglion']

for i in range(len(models)):
    models[k[i]].id = models_names[i]

In [2]:
# make RPE-PR model
# copy RPE, PR, and Human1 models
mod = models['Human-GEM.xml'].copy()
mod_RPE = models['model_06_MeanExpression_RPE__VoigtEtAl2019_.xml'].copy()
mod_PR = models['model_13_MeanExpression_RodPhotoreceptors__LiangEtAl_2019_.xml'].copy()

# add back any lost exchange reactions
def add_all_EX_rxns(basis_model,model):
    EX_rxns = [r for r in basis_model.reactions if len(r.products)==0]
    EX_rxns_model = [r.id for r in model.reactions if len(r.products)==0]
    missing_rxns = [r.copy() for r in EX_rxns if r.id not in EX_rxns_model]
    model.add_reactions(missing_rxns)
    return model
mod_RPE = add_all_EX_rxns(mod,mod_RPE)
mod_PR = add_all_EX_rxns(mod,mod_PR)

def remove_compartment(model, compartment):
# remove compartment from model
    rxns_n = [r.id for r in model.reactions if compartment in r.compartments]
    mets_n = [m for m in model.metabolites if compartment in m.compartment]
    model.remove_reactions(rxns_n)
    model.remove_metabolites(mets_n)
    return model

# remove nucleus from PR model
mod_PR = remove_compartment(mod_PR, 'n')

def add_id_suffix(model, suffix):
    for r in model.reactions:
        r.id = r.id + suffix
    for m in model.metabolites:
        m.id = m.id + suffix
        m.compartment = m.compartment + suffix
    return model
# add '_PR' to PR model IDs and '_RPE' to RPE model IDs
mod_PR = add_id_suffix(mod_PR, '_PR')
mod_RPE = add_id_suffix(mod_RPE, '_RPE')

C:\Users\prins\AppData\Roaming\Python\Python310\site-packages\cobra\core\group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")


In [3]:
# add RPE-PR interface reactions to RPE model (rxn_RPE-PR = RPE <--> RPE-PR interface)
for rxns in  [r for r in mod_RPE.reactions if 'e_RPE' in r.reaction]:     
    rxn = rxns.copy()
    rxn.id = rxn.id.replace('_RPE','_RPE-PR')
    for p in rxn.products:
        if 'e_RPE' in p.id:
            p.id = p.id.replace('e_RPE','e_RPE-PR')
            p.compartment = 'e_RPE-PR'
    for r in rxn.reactants:
        if 'e_RPE' in r.id:
            r.id = r.id.replace('e_RPE','e_RPE-PR')
            r.compartment = 'e_RPE-PR'
    mod_RPE.add_reactions([rxn])

In [4]:
# add RPE-PR interface reactions to PR model (rxn_PR-RPE = PR <--> RPE-PR interface)
for rxn in [r for r in mod_PR.reactions if 'e_PR' in r.reaction]:
    rxn.id = rxn.id.replace('_PR','_PR-RPE')
    for p in rxn.products:
        if 'e_PR' in p.id:
            p.id = p.id.replace('e_PR','e_RPE-PR')
            p.compartment = 'e_RPE-PR'
    for r in rxn.reactants:
        if 'e_PR' in r.id:
            r.id = r.id.replace('e_PR','e_RPE-PR')
            r.compartment = 'e_RPE-PR'

In [5]:
# fuse RPE and PR model
from cobra import Model
mod_RPE_PR = Model('RPE-PR model')
mod_RPE_PR.add_reactions(mod_RPE.reactions)
mod_RPE_PR.add_reactions(mod_PR.reactions)

In [6]:
# list reaction IDs to be deleted (because their reactions are duplicated)
import pandas as pd
df = pd.DataFrame([[r.id, r.reaction] for r in mod_RPE_PR.reactions \
                   if r.compartments == {'e_RPE-PR'}], columns=['id','reaction'])
l  = list(df['id'][df['reaction'].duplicated()])

# delete duplicated reactions and change reaction IDs (suffix: _eRPE-PR) 
# of the reactions that only involve the e_RPE-PR compartment (interface)
for r in [r for r in mod_RPE_PR.reactions if r.compartments=={'e_RPE-PR'}]: 
    if r.id in l:
        mod_RPE_PR.remove_reactions(r.id)
    elif '_RPE-PR' in r.id:
        r.id = r.id.replace('_RPE-PR','_eRPE-PR')
    elif '_PR-RPE' in r.id:
        r.id = r.id.replace('_PR-RPE','_eRPE-PR')
        
def fix_compartment_dict(model):
    new_compartment_dict = model.compartments.copy()
    for k in list(new_compartment_dict.keys()):
        new_compartment_dict[k] = k 
    model.compartments = new_compartment_dict
    return model
fix_compartment_dict(mod_RPE_PR)

C:\Users\prins\AppData\Roaming\Python\Python310\site-packages\cobra\core\model.py:778: UserWarning: need to pass in a list
  warn("need to pass in a list")


In [77]:
# add ATP demand reaction (atp[c] -> )
# met IDs are MAM01371c_PR and MAM01371c_RPE

def make_reaction(r_id,r_name,r_subsystem,r_lowerbound,r_upperbound):
    from cobra import Reaction
    r = Reaction(r_id)
    r.name = r_name
    r.subsystem = r_subsystem
    r.lower_bound = r_lowerbound  
    r.upper_bound = r_upperbound  
    return r

# add ATP demand reactions
DM_atp_c_RPE = make_reaction('DM_atp_c_RPE', 'ATP demand', 'Demand', -1000,1000)
DM_atp_c_PR = make_reaction('DM_atp_c_PR', 'ATP demand', 'Demand', -1000,1000)
DM_atp_c_RPE.add_metabolites({mod_RPE_PR.metabolites.get_by_id('MAM01371c_RPE'): -1.0,})
DM_atp_c_PR.add_metabolites({mod_RPE_PR.metabolites.get_by_id('MAM01371c_PR'): -1.0,})
mod_RPE_PR.add_reactions([DM_atp_c_RPE, DM_atp_c_PR])

# save RPE-PR model
from cobra.io import write_sbml_model
write_sbml_model(mod_RPE_PR, "mod_RPE_PR.xml")

Ignoring reaction 'DM_atp_c_RPE' since it already exists.
Ignoring reaction 'DM_atp_c_PR' since it already exists.


In [19]:
# close all exchange reactions
for r in [r for r in mod_RPE_PR.reactions if len(r.products)==0]: r.bounds = (0,0)
    
# open ATP demand reactions
mod_RPE_PR.reactions.get_by_id('DM_atp_c_RPE').bounds=(0,1000)
mod_RPE_PR.reactions.get_by_id('DM_atp_c_PR').bounds=(0,1000)

In [9]:
mod_RPE_PR.reactions.get_by_id('MAR09034_eRPE-PR')

Reaction identifier,MAR09034_eRPE-PR
Name,
Memory address,0x22ab640c9a0
Stoichiometry,MAM01965e_RPE-PR --> glucose -->
GPR,
Lower bound,0
Upper bound,0


In [15]:
# import excel file with info on exchange upper bounds 
# file contains 'id' col with IDs and an 'upper_bound' col with upper bounds to modify
bounds = pd.read_excel(folder / 'rxn_bounds' / 'R3D301_EX_rxns_RPE-PR.xlsx')
# select entries with non-zero upper bounds
bounds = bounds[bounds['upper_bound']>0]
# set bounds Exchange reactions (lower bounds -1000, upper bounds based on file info)
for i in range(len(bounds)):
    mod_RPE_PR.reactions.get_by_id(bounds['id'][i]).bounds = (-bounds['upper_bound'][i],1000)

In [17]:
mod_RPE_PR.reactions.get_by_id('MAR09034_RPE')

Reaction identifier,MAR09034_RPE
Name,
Memory address,0x22ab1a92fe0
Stoichiometry,MAM01965e_RPE <=> glucose <=>
GPR,
Lower bound,-40.0
Upper bound,1000


In [16]:
mod_RPE_PR.reactions.get_by_id('MAR09034_eRPE-PR')

Reaction identifier,MAR09034_eRPE-PR
Name,
Memory address,0x22ab640c9a0
Stoichiometry,MAM01965e_RPE-PR <=> glucose <=>
GPR,
Lower bound,-0.0004
Upper bound,1000


In [20]:
mod_RPE_PR.reactions.get_by_id('DM_atp_c_PR')


Reaction identifier,DM_atp_c_PR
Name,ATP demand
Memory address,0x22ab93e46a0
Stoichiometry,MAM01371c_PR --> ATP -->
GPR,
Lower bound,0
Upper bound,1000


In [27]:
# OBJECTIVE 
# set maximizing the ATP demand in PR as objective
# while constraining it to be twice the ATP demand in the RPE

mod_RPE_PR.objective = mod_RPE_PR.reactions.get_by_id('DM_atp_c_RPE').flux_expression
mod_RPE_PR.objective.expression

1.0*DM_atp_c_RPE - 1.0*DM_atp_c_RPE_reverse_457d3

In [28]:
from cobra.flux_analysis.loopless import add_loopless, loopless_solution
from cobra.flux_analysis import pfba
solution = mod_RPE_PR.optimize()
print(solution)
mod_RPE_PR.summary()

<Solution 0.000 at 0x22ac0ad5de0>


Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


In [ ]:
# with mod_RPE_PR:
#    add_loopless(mod_RPE_PR)
#    try: solution = mod_RPE_PR.optimize()
#    except: print('model is infeasible')
mod_RPE_PR.summary()

In [ ]:
# FVA
#from cobra.flux_analysis import flux_variability_analysis
#flux_variability_analysis(mod_RPE_PR,loopless=True)

In [ ]:
# list all exchange and demand reactions matching Human1 and VMH IDs
def get_vmh_id(reaction):
    if 'vmhreaction' in reaction.annotation.keys(): vmh_id = reaction.annotation['vmhreaction']
    else: vmh_id = ''
    return vmh_id
df_EX_rxns = pd.DataFrame([[r.id,get_vmh_id(r),r.name,r.reaction,r.build_reaction_string(use_metabolite_names = True)] \
              for r in mod.reactions if len(r.products) == 0],columns=['id','vmh id','name','reaction','reaction (names)'])
df_EX_rxns

In [ ]:
# ATP metabolites
ATP_mets = [[met.id,met.name] for met in mod_RPE_PR.metabolites if met.name=='ATP']
pd.DataFrame(ATP_mets,columns=['ID','name'])

# ATP reactions
ATP_rxns = [[r.id,get_vmh_id(r),r.name,r.subsystem,r.reaction,r.build_reaction_string(use_metabolite_names = True)] \
            for r in mod_RPE_PR.reactions if 'MAM01371' in r.reaction]

pd.DataFrame(ATP_rxns,columns=['id','vmh id', 'name', 'subsystem', 'reaction (id)', 'reaction (name)'])

In [ ]:
# list all exchange and demand reactions
df_EX_rxns_human1 = pd.DataFrame([[r.id,get_vmh_id(r),r.name,r.reaction,r.build_reaction_string(use_metabolite_names = True)] \
              for r in mod.reactions if len(r.products)==0],columns=['id','vmh id','name','reaction','reaction (names)'])
df_EX_rxns.to_clipboard(excel=True, sep=None)
df_EX_rxns

In [ ]:
# match VMH_IDs from bounds file to Human1 IDs to create 'R3D301_EX_rxns_RPE-PR.xlsx'

# import excel file with info on bounds
bounds = pd.read_excel(folder / 'rxn_bounds' / 'files PL' / 'R3D301_EX_rxns.xlsx')

# vmh_id lookup table + look up RPE rxn
df_vmh_id = pd.DataFrame([[get_vmh_id(r),r.id,r.id+'_RPE'] for r in mod.reactions if 'EX_' in get_vmh_id(r)],columns=['vmh_id','human1_id','id'])
merged_df = df_vmh_id.merge(bounds, how = 'right', on = ['vmh_id'])
merged_df.to_clipboard(excel=True, sep=None)
merged_df

In [ ]:
# biomass reaction
mod.reactions.get_by_id('MAR04413') 

pd.DataFrame([[r.id,r.name,r.reaction,r.reaction,r.build_reaction_string(use_metabolite_names = True)]\
              for r in mod_RPE_PR.reactions if 'biomass' in r.name])

In [ ]:
mod.reactions.get_by_id('MAR13082') 